In [2]:
import pandas as pd

In [3]:
data = pd.read_csv("verified_online.csv")
data.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,6935776,http://aveceditora.com.br/knxtkbeyuf.html?gcGD...,http://www.phishtank.com/phish_detail.php?phis...,2021-01-25T10:59:44+00:00,yes,2021-01-25T11:05:25+00:00,yes,Other
1,6935770,http://seemtechnosys.com/rxfawolwnn.html?gcGDR...,http://www.phishtank.com/phish_detail.php?phis...,2021-01-25T10:59:36+00:00,yes,2021-01-25T11:02:38+00:00,yes,Other
2,6935772,http://35.185.183.99:80/wordpress/wp-content/t...,http://www.phishtank.com/phish_detail.php?phis...,2021-01-25T10:59:36+00:00,yes,2021-01-25T11:03:32+00:00,yes,Other
3,6935757,https://epicindiagroup.com/1/Ds5c/,http://www.phishtank.com/phish_detail.php?phis...,2021-01-25T10:47:50+00:00,yes,2021-01-25T10:51:45+00:00,yes,Other
4,6935753,https://trskl.com.br/office36555/office365-Aut...,http://www.phishtank.com/phish_detail.php?phis...,2021-01-25T10:39:40+00:00,yes,2021-01-25T10:44:54+00:00,yes,Other


In [4]:
data.shape

(10043, 8)

A dataset containing 5500 URLs will be considered out of this datset as the source is updated frequently and could lead to data imbalance

In [5]:
# selecting 5500 samples from phishing datset
phishurl = data.sample(n = 5500, random_state = 14).copy()
phishurl = phishurl.reset_index(drop=True)
phishurl.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,6557141,https://betasus111.blogspot.com/,http://www.phishtank.com/phish_detail.php?phis...,2020-05-09T23:57:08+00:00,yes,2020-05-10T05:58:17+00:00,yes,Other
1,6248016,https://xag42asz.appspot.com/,http://www.phishtank.com/phish_detail.php?phis...,2019-10-21T19:18:12+00:00,yes,2019-10-21T19:19:38+00:00,yes,Google
2,6713913,https://theumashow.com/new.recent-activity/sur...,http://www.phishtank.com/phish_detail.php?phis...,2020-08-02T02:04:19+00:00,yes,2020-08-02T02:11:30+00:00,yes,Other
3,6912780,http://amguevara.com/images/bl/ali/login.php,http://www.phishtank.com/phish_detail.php?phis...,2021-01-06T02:08:50+00:00,yes,2021-01-06T02:11:24+00:00,yes,Other
4,6889750,https://4vkjkwex22wbmemxbmimva-on.drv.tw/thrtn...,http://www.phishtank.com/phish_detail.php?phis...,2020-12-14T08:45:54+00:00,yes,2020-12-14T08:47:14+00:00,yes,Other


In [6]:
phishurl.shape

(5500, 8)

In [34]:
data1 = pd.read_csv("Benign_list_big_final.csv") #this dataset was obtained form the Canadian institue of Cybersecurity
data1.columns = ['URLs']
data1.head()

,URLs
0,http://1337x.to/torrent/1110018/Blackhat-2015-...
1,http://1337x.to/torrent/1122940/Blackhat-2015-...
2,http://1337x.to/torrent/1124395/Fast-and-Furio...
3,http://1337x.to/torrent/1145504/Avengers-Age-o...
4,http://1337x.to/torrent/1160078/Avengers-age-o...


In [35]:
legiurl = data1.sample(n = 5500, random_state = 14).copy()
legiurl = legiurl.reset_index(drop=True)
legiurl.head()

,URLs
0,http://censor.net.ua/video_news/335293/epopeya...
1,https://paytm.com/offer/blog/faq/got-a-better-...
2,http://sfglobe.com/2015/02/23/american-sniper-...
3,https://twitter.com/home?status=%E3%83%8C%E3%8...
4,http://torrentdn.com/bbs/s.php?bo_table=torren...


Feature Extraction:

In this step, features are extracted from the URLs dataset.

The extracted features are categorized into

Address Bar based Features

Address Bar Based Features:

Many features can be extracted that can be consided as address bar base features. Out of them, below mentioned were considered for this project.

Domain of URL

IP Address in URL

"@" Symbol in URL

Length of URL

Depth of URL

Redirection "//" in URL

"http/https" in Domain name

Using URL Shortening Services “TinyURL”

Prefix or Suffix "-" in Domain

In [36]:
from urllib.parse import urlparse,urlencode
import ipaddress
import re


3.1.1. Domain of the URL
Here, we are just extracting the domain present in the URL. This feature doesn't have much significance in the training. May even be dropped while training the model.

In [37]:
def getDomain(url):  
  domain = urlparse(url).netloc
  if re.match(r"^www.",domain):
       domain = domain.replace("www.","")
  return domain

 IP Address in the URL
Checks for the presence of IP address in the URL. URLs may have IP address instead of domain name. If an IP address is used as an alternative of the domain name in the URL, we can be sure that someone is trying to steal personal information with this URL.

If the domain part of URL has IP address, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [38]:
def havingIP(url):
  try:
    ipaddress.ip_address(url)
    ip = 1
  except:
    ip = 0
  return ip

"@" Symbol in URL
Checks for the presence of '@' symbol in the URL. Using “@” symbol in the URL leads the browser to ignore everything preceding the “@” symbol and the real address often follows the “@” symbol.

If the URL has '@' symbol, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [39]:
def haveAtSign(url):
  if "@" in url:
    at = 1    
  else:
    at = 0    
  return at

Length of URL
Computes the length of the URL. Phishers can use long URL to hide the doubtful part in the address bar. In this project, if the length of the URL is greater than or equal 54 characters then the URL classified as phishing otherwise legitimate.

If the length of URL >= 54 , the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [40]:
def getLength(url):
  if len(url) < 54:
    length = 0            
  else:
    length = 1            
  return length

Depth of URL
Computes the depth of the URL. This feature calculates the number of sub pages in the given url based on the '/'.

The value of feature is a numerical based on the URL.

In [41]:
def getDepth(url):
  s = urlparse(url).path.split('/')
  depth = 0
  for j in range(len(s)):
    if len(s[j]) != 0:
      depth = depth+1
  return depth

Redirection "//" in URL
Checks the presence of "//" in the URL. The existence of “//” within the URL path means that the user will be redirected to another website. The location of the “//” in URL is computed. We find that if the URL starts with “HTTP”, that means the “//” should appear in the sixth position. However, if the URL employs “HTTPS” then the “//” should appear in seventh position.

If the "//" is anywhere in the URL apart from after the protocal, thee value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [42]:
def redirection(url):
  pos = url.rfind('//')
  if pos > 6:
    if pos > 7:
      return 1
    else:
      return 0
  else:
    return 0

 Using URL Shortening Services “TinyURL”
URL shortening is a method on the “World Wide Web” in which a URL may be made considerably smaller in length and still lead to the required webpage. This is accomplished by means of an “HTTP Redirect” on a domain name that is short, which links to the webpage that has a long URL.

If the URL is using Shortening Services, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [43]:

# 7.Existence of “HTTPS” Token in the Domain Part of the URL (https_Domain)
def httpDomain(url):
  domain = urlparse(url).netloc
  if 'https' in domain:
    return 1
  else:
    return 0

 Using URL Shortening Services “TinyURL”
URL shortening is a method on the “World Wide Web” in which a URL may be made considerably smaller in length and still lead to the required webpage. This is accomplished by means of an “HTTP Redirect” on a domain name that is short, which links to the webpage that has a long URL.

If the URL is using Shortening Services, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [44]:
shortening_services = r"bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|" \
                      r"yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|" \
                      r"short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|" \
                      r"doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|db\.tt|" \
                      r"qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|q\.gs|is\.gd|" \
                      r"po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|x\.co|" \
                      r"prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|" \
                      r"tr\.im|link\.zip\.net"

In [45]:

# 8. Checking for Shortening Services in URL (Tiny_URL)
def tinyURL(url):
    match=re.search(shortening_services,url)
    if match:
        return 1
    else:
        return 0

 Prefix or Suffix "-" in Domain
Checking the presence of '-' in the domain part of URL. The dash symbol is rarely used in legitimate URLs. Phishers tend to add prefixes or suffixes separated by (-) to the domain name so that users feel that they are dealing with a legitimate webpage.

If the URL has '-' symbol in the domain part of the URL, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [46]:
def prefixSuffix(url):
    if '-' in urlparse(url).netloc:
        return 1            # phishing
    else:
        return 0            # legitimate

 Computing URL Features
Create a list and a function that calls the other functions and stores all the features of the URL in the list. We will extract the features of each URL and append to this list.

In [47]:
#Function to extract features
def featureExtraction(url,label):

  features = []
  #Address bar based features (10)
  features.append(getDomain(url))
  features.append(havingIP(url))
  features.append(haveAtSign(url))
  features.append(getLength(url))
  features.append(getDepth(url))
  features.append(redirection(url))
  features.append(httpDomain(url))
  features.append(tinyURL(url))
  features.append(prefixSuffix(url))
  features.append(label)
  
  return features

In [48]:
legiurl.shape

(5500, 1)

In [49]:
#Extracting the feautres & storing them in a list
legi_features = []
label = 0

for i in range(0, 5500):
  url = legiurl['URLs'][i]
  legi_features.append(featureExtraction(url,label))


In [50]:
#converting the list to dataframe
feature_names = ['Domain', 'Have_IP', 'Have_At', 'URL_Length', 'URL_Depth','Redirection','https_Domain', 'TinyURL', 'Prefix/Suffix','Label']



In [51]:

legitimate = pd.DataFrame(legi_features, columns= feature_names)
legitimate.head()

,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,Label
0,censor.net.ua,0,0,1,3,0,0,0,0,0
1,paytm.com,0,0,1,4,0,0,0,0,0
2,sfglobe.com,0,0,1,4,0,0,0,0,0
3,twitter.com,0,0,1,1,0,0,0,0,0
4,torrentdn.com,0,0,1,2,0,0,0,0,0


In [81]:
legitimate.to_csv('legitimate.csv', index= False)
#storing it to a legitimate dataset called legitimate.csv

In [27]:
#Extracting the feautres & storing them in a list
phish_features = []
label = 1
for i in range(0, 5500):
  url = phishurl['url'][i]
  phish_features.append(featureExtraction(url,label))

#converting the list to dataframe
feature_names = ['Domain', 'Have_IP', 'Have_At', 'URL_Length', 'URL_Depth','Redirection', 
                      'https_Domain', 'TinyURL', 'Prefix/Suffix','Label']

phishing = pd.DataFrame(phish_features, columns= feature_names)
phishing.head()

,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,Label
0,betasus111.blogspot.com,0,0,0,0,0,0,1,0,1
1,xag42asz.appspot.com,0,0,0,0,0,0,1,0,1
2,theumashow.com,0,0,0,2,0,0,0,0,1
3,amguevara.com,0,0,0,4,0,0,0,0,1
4,4vkjkwex22wbmemxbmimva-on.drv.tw,0,1,1,2,0,0,0,1,1


In [28]:
#converting it to csv
phishing.to_csv('phishing.csv', index= False)

In [29]:
urldata = pd.concat([legitimate, phishing]).reset_index(drop=True)


In [30]:
urldata.shape

# Storing the data in CSV file
urldata.to_csv('urldata.csv', index=False)

In [3]:
#unimportant 
import tensorflow 
from tensorflow import keras
keras.__version__

'2.2.4-tf'